# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

## Setting up your keys

If you haven't done so already, you'll need to create API keys from OpenAI, Anthropic and Google.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables.

EITHER (recommended) create a file called `.env` in this project root directory, and set your keys there:

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

OR enter the keys directly in the cells below.

You may need to restart the Jupyter Notebook Kernel (the Python process) via the Kernel menu.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [3]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
# Having problems with API files? You can use openai = OpenAI(api_key="your-key-here") and same for claude
# Having problems with Google Gemini setup? Then just skip Gemini; you'll get all the experience you need from GPT and Claude.

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [4]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [5]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [6]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why was the statistics book always so calm?
Because it had a lot of confidence intervals!


In [7]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they wanted to reach new heights in their analysis!


In [8]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they heard the cloud was high up!


In [9]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much multicollinearity in their relationship!

(Explanation for non-data scientists: Multicollinearity is a statistical concept where two or more variables in a model are highly correlated, making it difficult to determine their individual effects. In relationships, it might imply that the couple was too similar or their lives were too intertwined!)


In [11]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist become a gardener?

They wanted to work with real-world arrays!

Ba dum tss! 🥁

This joke plays on the double meaning of "arrays" - in programming, arrays are data structures, while in gardening, they refer to organized rows of plants. It's a bit of wordplay that data scientists might appreciate!

In [12]:
# The API for Gemini has a slightly different structure

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why did the data scientist break up with the statistician? 

Because they couldn't see eye to eye on the p-value! 



In [13]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [14]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Deciding if a business problem is suitable for a Large Language Model (LLM) solution involves several considerations. Here are some steps and factors to guide your decision:

### 1. **Problem Nature and Complexity**
   - **Text-Intensive Tasks**: LLMs are well-suited for tasks involving natural language understanding, generation, or transformation. Examples include summarization, translation, sentiment analysis, and content creation.
   - **Unstructured Data**: If your problem involves unstructured text data, LLMs might be appropriate as they excel at extracting information and insights from such data.
   - **Complex Language Tasks**: For tasks requiring nuanced understanding of language, such as legal document analysis or customer service automation, LLMs can be beneficial.

### 2. **Data Availability**
   - **Sufficient Quality Data**: LLMs require a large volume of high-quality text data to perform well. Ensure you have access to relevant data for training or fine-tuning the model.
   - **Data Privacy and Security**: Consider the sensitivity of your data. Using LLMs may involve sharing data with third-party services, which raises privacy and compliance issues.

### 3. **Performance Requirements**
   - **Accuracy and Precision**: Evaluate if the LLM can meet the accuracy and precision requirements of your task. Some tasks may require deterministic outputs that current LLMs may not consistently provide.
   - **Response Time**: LLMs can be computationally expensive and may not meet real-time processing needs without significant infrastructure.

### 4. **Cost-Benefit Analysis**
   - **Implementation Costs**: Assess the costs associated with deploying an LLM, including model training, fine-tuning, and infrastructure.
   - **Value Addition**: Consider the potential ROI. Will the LLM solution add significant value or efficiency to your business processes?

### 5. **Ethical and Legal Considerations**
   - **Bias and Fairness**: LLMs may exhibit biases present in their training data. Assess the impact of these biases on your business and its stakeholders.
   - **Compliance**: Ensure that using an LLM complies with relevant regulations and standards, such as GDPR for data protection.

### 6. **Technical Expertise and Resources**
   - **In-House Expertise**: Determine if your team has the necessary skills to implement, manage, and maintain an LLM solution.
   - **External Support**: If not, consider the availability of external vendors or consultants who can provide the needed expertise.

### 7. **Integration with Existing Systems**
   - **Compatibility**: Evaluate how well an LLM solution can integrate with your existing IT infrastructure and workflows.
   - **Interoperability**: Ensure that the outputs of the LLM can be effectively used by other systems within your organization.

### Conclusion
To decide if an LLM is suitable for your business problem, carefully analyze the nature of the problem, data requirements, performance expectations, cost implications, ethical considerations, and your organization's technical capacity. Conducting pilot tests or proof-of-concept projects can also be beneficial in assessing the viability of an LLM solution for your specific needs.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

# See the community-contributions folder

For a great variation with a 3-way bringing Gemini into the conversation!